In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import json
import numpy as np
import sys
sys.path.insert(0, "../")

In [2]:
import openai

import os
from getpass import getpass

if 'OPENAI_API_KEY' not in os.environ:
    print("You didn't set your OPENAI_API_KEY on the command line.")
    os.environ['OPENAI_API_KEY'] = getpass("Please enter your OpenAI API Key: ")

You didn't set your OPENAI_API_KEY on the command line.


## Trialing stuff

In [ ]:


with open("../assets/character_traits.json", 'r') as f:
    characters = json.load(f)

In [ ]:
def get_text_embedding(client, text, model="text-embedding-3-small"):
    text_vector = client.embeddings.create(input=[text], model=model).data[0].embedding
    return text_vector

In [ ]:
embedded_characters = {}
client = openai.Client()
for i, c in enumerate(characters):
    c_vec = get_text_embedding(client, c.__str__())
    embedded_characters[i] = c_vec

In [ ]:

def find_similar_character(query, characters, top_n=1):   
    client = openai.Client() 
    query_vec = np.array(get_text_embedding(client, query))

    sim = cosine_similarity(query_vec.reshape(1, -1), np.array([np.array(v) for v in characters.values()]))

    idx = sorted(enumerate(sim[0]), key=lambda x: x[1], reverse=True)[:top_n][0][0]

    return idx

In [ ]:
idx = find_similar(query="a barista from the southwestern United States",
             characters=embedded_characters,
             top_n=1)

In [ ]:
characters[idx]

In [ ]:
import numpy as np
lst = [10, 20, 30, 100, -39]
lst.extend(np.random.randint(0, 100, 3).tolist())

In [ ]:
np.clip(lst, 0, 100)

In [ ]:
from text_adventure_games.scales.base import BaseScale

test_scale = BaseScale(("Prejudicial", "Unbiased"))

In [ ]:
print(test_scale)

In [ ]:
MONITORED_TRAITS = {
        "judgement": ("Prejudicial", "Unbiased"),
        "cooperation": ("Stubborn", "Agreeable"),
        "outlook": ("Pessimistic", "Optimistic"),
        "initiative": ("Passive", "Assertive"),
        "generosity": ("Selfish", "Generous"),
        "social": ("Follower", "Learder"),
        "mind": ("Creative", "Logical"),
        "openness": ("Close-minded", "Open-minded"),
        "stress": ("Anxious", "Calm")
    }

scores = [i for i in range(10)]

In [ ]:
print(*zip(MONITORED_TRAITS.items(), scores))

In [ ]:
import os
import json
assets_path = "../assets"
with open(os.path.join(assets_path, "persona_profiles.json"), "r") as pfs:
    profiles = json.load(pfs)

In [ ]:
archs = profiles['archetypes']

In [ ]:
archs[0]

In [ ]:
for a in archs:
    for i, trait in enumerate(a["traits"]):
        trait.update({"name": names[i]})

## Test Persona Creation

In [ ]:
# imports
from text_adventure_games import agent
from text_adventure_games.utils import setup_agent, general, gpt

### Test Get Existing Facts

In [ ]:
facts = setup_agent.get_or_create_base_facts("a barista from the united states", make_new=False)
assert facts == {'Name': 'Jacob Harrison',
 'Age': 25,
 'Likes': ['coffee brewing', 'indie music', 'baking', 'dogs', 'reading'],
 'Dislikes': ['rude customers',
  'early mornings',
  'negativity',
  'instant coffee'],
 'Occupation': 'Barista',
 'Home city': 'Philadelphia, Pennsylvania'}

### Test Get new Facts

In [ ]:
facts = setup_agent.get_or_create_base_facts("A scuba diver with a house boat", make_new=True)

In [ ]:
goals = {"flex": "Spend time in the ocean"}
facts = {'Name': 'Jacob Harrison',
 'Age': 25,
 'Likes': ['coffee brewing', 'indie music', 'baking', 'dogs', 'reading'],
 'Dislikes': ['rude customers',
  'early mornings',
  'negativity',
  'instant coffee'],
 'Occupation': 'Barista',
 'Home city': 'Philadelphia, Pennsylvania'}

### Test get archetype

In [ ]:
target_profile = general.get_archetype_profiles("Hubris")

### Creat a Persona

In [ ]:
p = setup_agent.create_persona(facts=facts, goals=goals, archetype="Hubris")

## Spacy parsing of game commands

In [5]:
import spacy
from collections import defaultdict

In [6]:
nlp = spacy.load('en_core_web_sm')

In [4]:
def extract_key_deps(text):
    characters = {"troll" : "a mean troll",
                  "player": "the main characater"}
    doc = nlp(text)
    keys = defaultdict(list)
    for w in doc:
        if "subj" in w.dep_:
            if w.text in characters:
                keys['characters'].append(w.text)
            else:
                keys['misc_deps'].append(w.text)
        if "obj" in w.dep_:
            if w.text in characters:
                keys['characters'].append(w.text)
            else:
                keys['objects'].append(w.text)
    return keys

In [7]:
import re
re.sub("the player", "You", "The troll attempted to give the fishing pole to the player, but failed.")

'The troll attempted to give the fishing pole to You, but failed.'

In [6]:
deps = extract_key_deps("From Cottage The player could go out to Garden Path")
deps

defaultdict(list, {'objects': ['Cottage', 'Path'], 'characters': ['player']})

In [8]:
s = 'location: The player is at A small cottage.\nexits: From Cottage The player could go: Out to Garden Path, \nitems: \ncharacters: The player sees characters: A motherly figure, \n'
re.split('\n+', s)

['location: The player is at A small cottage.',
 'exits: From Cottage The player could go: Out to Garden Path, ',
 'items: ',
 'characters: The player sees characters: A motherly figure, ',
 '']

In [9]:
def parse_location_description(text):
    by_description_type = re.split('\n+', text)
    new_observations = {}
    if len(by_description_type) >= 1:
        desc_type, loc = map(lambda x: x.strip(), by_description_type[0].split(":"))
        new_observations[desc_type] = loc
        for description in by_description_type[1:]:
            if description:
                try: 
                    desc_type, player, observed = map(lambda x: x.strip(), description.split(":"))
                except ValueError:
                    # Likely not enough values to unpack
                    desc_type = description.split(":")[0]
                    new_observations[desc_type] = f'No {desc_type} here'
                    continue
                if "(" in observed:
                    new_observations[desc_type] = [f"{player} {obs}" for obs in observed.split(';') if obs]
                else:
                    new_observations[desc_type] = [f"{player} {obs}" for obs in observed.split(',') if obs]
    return new_observations

In [10]:
parse_location_description(s)

{'location': ' The player is at A small cottage.',
 'exits': [' From Cottage The player could go  Out to Garden Path',
  ' From Cottage The player could go  '],
 'items': 'No items here',
 'characters': [' The player sees characters  A motherly figure',
  ' The player sees characters  ']}

## Test GPT-Parser Integration

In [2]:
import sys
import openai
sys.path.insert(0, "../..")
import action_castle
from SurvivorWorld.text_adventure_games.parsing import GptParser3
from SurvivorWorld.text_adventure_games.things import Character

sk-27...
sk-27...


In [3]:
#game = action_castle.build_game(type=2)
game = action_castle.build_mini_game()
game.give_hints = True
parser = GptParser3(game, verbose=False, model='gpt-4')
game.set_parser(parser)
parser.refresh_command_list()

Adding custom actions
sk-27...


In [4]:
try:
    game.game_loop()
except openai.APIConnectionError:
    print("Could not connect to OpenAI API")

ROUND: 0.0
Character: The player (id: 7)
location: The player is at A small cottage.
exits: From Cottage The player could go: Out to Garden Path, 
items: 
characters: The player sees characters: A motherly figure, 

['location: The player is at A small cottage.\nexits: From Cottage The player could go: Out to Garden Path, \nitems: \ncharacters: The player sees characters: A motherly figure, \n']
Chosen Command: describe


AttributeError: 'list' object has no attribute 'add_memory'